# Dev 2

We've come along way with this project. Now we are thoroughly in the weeds. 

Our major goasls are to:
- make various updates to make it easier to scale with this project. Mainly storing artifacts (trajectories/models) on wandb. Once this is working, we can train on a lambda labs gpu on harder problems while having a complete record of events and results.
- next, I'd like to write more complicated analysis code to characterise various models. For starter, I'd like to get a calibration curve to evaluate DT's on their ability to "simulate" different qualities of agents. 

Side goals which I may do in parallel:
- abstract away from the DT to also have a BC model. 
- train the DT on predicting rewards, states and actions (this may improve sample efficiency and is fine from the perspective of the interpretability project). 



## Downloading artifacts from wandb

In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('arena-ldn/PPO-MiniGrid/MiniGrid-LavaGapS7-v0016df0e6-b5e6-40ed-a598-e75f228f45a8.pkl:v0', type='trajectory')
artifact_dir = artifact.download()

wandb: Currently logged in as: jbloomaus (arena-ldn). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


now I'll trigger the DT training script with this path.

It seems like the upload to wandb takes ages... possibly due to an unresolved issue. 

I will hold off on scaling up/using lambda while I try to improve on the DT, mainly by making it predict RTG and States as well.

# Investigate Trajectories

Things it would be good to control:
- train test split
- the reward per trajectory
- epochs
- batch size
- mask -> should we mask the padding tokens?

In [1]:
# let's load some trajectories 
from src.decision_transformer.offline_dataset import TrajectoryLoader
from src.decision_transformer.decision_transformer import DecisionTransformer
from src.environments import make_env

trajectory_path = "trajectories/MiniGrid-DoorKey-8x8-v0c96fb2f4-23b1-4d9e-af06-906fd108c700.pkl"
# trajectory_path = "trajectories/MiniGrid-Dynamic-Obstacles-8x8-v0579f73fd-a62e-4239-a679-55fd341c23c7.pkl"
trajectory_data_set = TrajectoryLoader(trajectory_path, pct_traj=1, device="cpu")

env_id = trajectory_data_set.metadata['args']['env_id']
env = make_env(env_id, seed = 1, idx = 0, capture_video=False, run_name = "dev", fully_observed=False)
env = env()

# dt = DecisionTransformer(
#     env = env, 
#     d_model = 32,
#     n_heads = 2,
#     d_mlp = 64,
#     n_layers = 2,
#     state_embedding_type="grid", # hard-coded for now to minigrid.
#     max_timestep=trajectory_data_set.metadata.get("args").get("max_steps") # Our DT must have a context window large enough
# )



{'args': {'exp_name': 'MiniGrid-DoorKey-8x8-v0', 'seed': 1, 'cuda': True, 'track': True, 'wandb_project_name': 'PPO-MiniGrid', 'wandb_entity': None, 'capture_video': True, 'env_id': 'MiniGrid-DoorKey-8x8-v0', 'total_timesteps': 1000000, 'learning_rate': 0.00025, 'num_envs': 4, 'num_steps': 128, 'gamma': 0.99, 'gae_lambda': 0.95, 'num_minibatches': 4, 'update_epochs': 4, 'clip_coef': 0.2, 'ent_coef': 0.01, 'vf_coef': 0.5, 'max_grad_norm': 0.5, 'max_steps': 1000, 'trajectory_path': 'trajectories/MiniGrid-DoorKey-8x8-v0c96fb2f4-23b1-4d9e-af06-906fd108c700.pkl', 'fully_observed': False}, 'time': 1673097011.91648}


In [10]:
state, action, reward, d, rtg, timesteps, mask = trajectory_data_set.get_batch(256, 100)


In [41]:
action[0][mask[0].to(bool)]

tensor([1., 5., 2., 2., 2., 2., 1., 2., 2., 2.])

In [42]:
reward[0][mask[0].to(bool)]

tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.9856]])

In [39]:
from minigrid.core.constants import IDX_TO_OBJECT
import numpy as np
import torch

def find_agent(observation):
    height = observation.shape[0]
    width = observation.shape[1]
    for i in range(width):
        for j in range(height):
            object = IDX_TO_OBJECT[int(observation[i,j][0])]
            if object == 'agent':
                return i, j

    else:
        return -1,-1


def render_minigrid_observation(env, observation):
    if isinstance(observation, np.ndarray):
        observation = observation.copy() # so we don't edit the original object
    elif isinstance(observation, torch.Tensor):
        observation = observation.numpy().copy()

    agent_pos = find_agent(observation)
    agent_dir = observation[agent_pos[0], agent_pos[1]][2]

    observation[agent_pos[0], agent_pos[1]] = [0,0,0]

    grid, _ = env.grid.decode(observation.astype(np.uint8))
    
    i = agent_pos[0]
    j = agent_pos[1]
    
    return grid.render(32, (i,j), agent_dir=agent_dir)

def render_minigrid_observations(env, observations):
    return np.array([render_minigrid_observation(env, observation) for observation in observations])



frames = render_minigrid_observations(env, state[0][mask[0].to(bool)])
import moviepy.editor as    mpy
import numpy as np
 
 # write an mp4 file
clip = mpy.ImageSequenceClip(list(frames), fps=10)
clip.write_videofile("test.mp4", fps=10)


Moviepy - Building video test.mp4.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
